# В конце этого семинара - домашка!

# PyTorch

## __Tensors__

Одно из основных понятий в PyTorch -- это __Tenosor__. 

https://pytorch.org/docs/master/tensors.html

__Tensor__ -- это такой же массив, как и в __numpy.array__, размерность и тип данных которого мы можем задать. Tensor в отличие от numpy.array может вычисляться на __GPU__.

In [2]:
!pip install torch

In [3]:
import numpy as np
import torch

In [4]:
N = 100
D_in = 50

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

x = np.random.randn(N, D_in)
x_torch = torch.randn(N, D_in, device=device, dtype=dtype)

In [5]:
x

array([[-0.93570063, -0.05293074,  0.39268223, ..., -0.3690987 ,
        -1.61434112, -0.38867744],
       [ 0.97547621,  1.260578  ,  1.71524162, ...,  0.18434539,
         0.027118  ,  0.71998177],
       [ 0.55758054,  0.56811361,  0.31764995, ...,  1.23173647,
        -0.42038149, -0.56099696],
       ...,
       [ 0.17593941,  1.04289383,  2.46617686, ..., -0.73155525,
        -1.72557655, -0.23307825],
       [-0.80008208,  1.18764565, -1.47807501, ...,  0.33762602,
         0.01668428,  1.47936854],
       [ 0.16195702, -0.26780082, -0.85710567, ...,  2.81437639,
        -0.0670021 , -0.67817159]])

In [6]:
x_torch

tensor([[ 0.5675,  1.0945,  2.2934,  ..., -0.1460, -0.5604,  0.1829],
        [ 0.5182,  0.5709,  1.2133,  ..., -0.9881, -0.9256,  0.4606],
        [-1.1364, -0.6727, -0.2791,  ..., -2.4274,  0.4110,  0.5120],
        ...,
        [-1.4661, -0.7049, -0.5312,  ...,  1.0933,  0.6811,  0.4306],
        [ 1.2689, -1.7790,  0.7693,  ...,  0.1270, -1.9487,  0.6724],
        [-0.7244,  0.1599,  2.0693,  ..., -0.1499, -0.0937, -0.0114]])

In [7]:
x_torch = torch.Tensor(np.ones((N, D_in)))
x_torch

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [8]:
x_torch = torch.FloatTensor([1, 2, 3])
x_torch

tensor([1., 2., 3.])

In [9]:
x1 = torch.IntTensor([1, 2, 3])
x2 = torch.FloatTensor([3, 4, 5])

In [10]:
print(x1)
print(x2)

tensor([1, 2, 3], dtype=torch.int32)
tensor([3., 4., 5.])


В PyTorch можно найти много операций, которые похожи на то, что есть в numpy :
```
- torch.add (np.add) -> сложение тензоров (поэлементное)
- torch.sub (np.subtract) -> вычитание (поэлементное)
- torch.mul (np.multiply) -> умнажение скаляров / матриц (поэлементное)
- torch.mm (np.matmul) -> перемножение матриц
- torch.ones (np.ones) -> создание тензора из единиц
```

In [15]:
# Давайте попробуем вышепересчисленные операции

In [11]:
x1 = torch.FloatTensor([[1, 2, 3], [4, 5, 6]])

In [12]:
x2 = torch.FloatTensor([[7, 8], [9, 1], [2, 3]])

In [13]:
out = torch.mm(x1, x2)
out

tensor([[31., 19.],
        [85., 55.]])

```
- torch.view (np.reshape) -> изменения порядка элементов в тензоре, не путать с транспонированием.
```

## Dynamic Computational Graph

После того, как были реализованы архитектура модели и весь процес обучения и валидация сети, при запуске кода в PyTorch происходят следующие этапы:

1. Строится вычислительный граф (направленный ациклический граф), где каждое ребро, ведущее к дргуому узлу, -- это тензор, а узел - это выполнение операции над данным тензором.

<img src="./images/Graph.png" alt="Drawing" style="width: 300px;"/>

Реализуем двухслойную сеть для задачи регрессии. И граф для такой архитектуры бдует выглядить следующим образом:

<img src="./images/RegGraph.png" alt="Drawing" />

In [14]:
batch_size = 64
input_size = 3
hidden_size = 2
output_size = 1

In [17]:
# Create random input and output data
x = torch.randn(batch_size, input_size, device=device, dtype=dtype)
y = torch.randn(batch_size, output_size, device=device, dtype=dtype)

# Randomly initialize weights
w1 = torch.randn(input_size, hidden_size, device=device, dtype=dtype)
w2 = torch.randn(hidden_size, output_size, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y
    #TODO
    h_1 = x.mm(w1)
    h_relu = h_1.clamp(min=0)
    out = h_relu.mm(w2)
    
    # Compute and print loss
    loss = (out - y).pow(2).sum().item()    
    
    
    # Backward pass:
    dloss_dout = 2 * (out - y)
    
    grad_w2 = h_relu.t().mm(dloss_dout)
    
    grad_h_relu = dloss_dout.mm(w2.t())
    
    grad_h_relu[h_1 < 0] = 0
    
    grad_w1 = x.t().mm(grad_h_relu)
    
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2
    if t % 100 == 99:
        print('Loss on iteration {} = {}'.format(t, loss))
    

Loss on iteration 99 = 111.7910385131836
Loss on iteration 199 = 110.04888153076172
Loss on iteration 299 = 108.39384460449219
Loss on iteration 399 = 106.82054138183594
Loss on iteration 499 = 105.3239974975586


In [18]:
loss

105.3239974975586

## Autograd

2. Еще одно фундаментальное понятие и важный элемент при построении графа -- это __Autograd__ -- автоматическое дифференцирование.

Для того чтобы с помощью стохастического градиентного спуска обновить обучаемые параметры сети, нужно посчитать градиенты. И как известно, обновление весов, которые учавтсвуют в нескольких операциях, происходит по `правилу дифференцирования сложной функции` (цепное правило или __chain rule__).

<img src="./images/RegChainRule.png" alt="Drawing" />

То есть (1) вычислительный граф позволяет определить последовательность операций, а (2) автоматическое дифференцирование посчитать нужные градиенты.

Если бы `Autograd` не было, то тогда backprop надо было бы реализовывать самим, и как это бы выглядело?

Рассмотрим на примере, как посчиать градиенты для весов из входного слоя, где входной вектора `X` состоит из 3-х компонент. А входной слой вторую размерность имеет равной 2. 

После чего это идет в `ReLU`, но для простоты опустим на время ее, и посмотрим как дальше это идет по сети.

Ниже написано, как это все вычисляется и приводит нас к значению целевой функции для одного наблюдения

<img src="./images/1.png" alt="1" style="width: 600px;"/>

Тогда, чтобы посчитать градиент по первому элементу из обучаемой матрицы на первом слое, необходимо взять производоную у сложной функции. А этот как раз делается по `chain rule`: сначала берем у внешней, потом спускаемся на уровень ниже, и так пока не додйдем до то функции, после которой эта перменная уже нигде не участвует:

<img src="./images/2.png" alt="2" style="width: 400px;"/>

Перепишем это все в матричном виде, то есть сделаем аналог вида матрицы весов из первого слоя, но там уже будут её градиенты, котоыре будут нужны чтобы как раз обновить эти веса:

<img src="./images/3.jpg" alt="3" style="width: 600px;"/>

Как видно, здесь можно вектор X вынести, то есть разделить на две матрицы:

<img src="./images/4.jpg" alt="4" style="width: 500px;"/>

То есть уже видно, что будем траспонировать входной вектор(матрицу). Но надо понимать, что в реальности у нас не одно наблюдение в батче, а несколько, тогда запись немного изменит свой вид:

<img src="./images/5.jpg" alt="5" style="width: 500px;"/>

Теперь мы видим, как на самом деле вычисляется вот те самые частные производные для вектора X, то есть видно, как математически это можно записать, а именно:

<img src="./images/6.jpg" alt="6" style="width: 500px;"/>

<img src="./images/7.jpg" alt="7" style="width: 500px;"/>

Уже можно реализовать. Понятно, что транспонируется, что нет, и что на что умножается.

Но помним про ReLU. Для простоты опустили, но теперь её учесть будет легче. 

Так как после первого слоя идет ReLU, а значит, занулились те выходы первого слоя, которые были __меньше__ нуля. Получается, что во второй слой не все дошло, тогда нужно обнулить, что занулил ReLU. 

Что занулил ReLU, мы можем выяснить при `forward pass`, а где именно поставить нули, то надо уже смотреть относительно `backward propagation`, на том выходе, где последний раз участвовал выход после ReLU, то есть:

<img src="./images/8.jpg" alt="8" style="width: 600px;"/>

Благодаря `Autograd` реализацию `chain rule` можно избежать, так как для более сложных нейронных сетей вручную такое реализовать сложно, при этом сделать это эффективным.

Для того чтобы PyTorch понял, за какими переменными надо "следить", то есть указать, что именно "эти" переменные являются обучаемыми, необходимо при создании тензора в качестве аттрибута указать __requires_grad=True__:

In [19]:
w1 = torch.randn(input_size, hidden_size, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(hidden_size, output_size, device=device, dtype=dtype, requires_grad=True)

In [20]:
learning_rate = 1e-6
for t in range(500):
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())
    
    # Теперь подсчет градиентов для весов происходит при вызове backward
    loss.backward()
   
    # Обновляем значение весов, но укзаываем, чтобы PyTorch не считал эту операцию, 
    # которая бы учавствовала бы при подсчете градиентов в chain rule
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # Теперь обнуляем значение градиентов, чтобы на следующем шаге 
        # они не учитывались при подсчете новых градиентов,
        # иначе произойдет суммирвоание старых и новых градиентов
        w1.grad.zero_()
        w2.grad.zero_()

99 121.06077575683594
199 119.01393127441406
299 117.0809555053711
399 115.25408935546875
499 113.5262222290039


Осталось еще не вручную обновлять веса, а использовать адаптивные методы градинетного спсука. Для этого нужно использовать модуль __optim__. А помимо оптимайзера, еще можно использовать готовые целевые функции из модлуя __nn__.

In [21]:
import torch.optim as optim

loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6
optimizer = torch.optim.Adam([w1, w2], lr=learning_rate)

for t in range(500):
    optimizer.zero_grad()
    
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
    
    loss.backward()
   
    optimizer.step()

99 113.4802474975586
199 113.4508056640625
299 113.42137145996094
399 113.39197540283203
499 113.3625717163086


После того, как мы сделали backward, в этот момент посчитались градиенты и граф уничтожился, то есть стёрлись все пути, которые связывали тензоры между собой. Это значит, что еще раз backward сделать не поулчится, будет ошибка. Но если вдруг нужно считать градиенты еще раз, то нужно при вызове backward задать `retain_graph=True`.

Еще важный аттрибут, который есть у Tensor -- это `grad_fn`. В этом аттрибуте указывается та функция, посредством которой был создан этот тензор. Так PyTorch понимает, как именно считать по нему градиент.

In [33]:
y_pred.grad_fn

Также можно контролировать, должны ли градиенты течь или нет.

In [37]:
x = torch.tensor([1.], requires_grad=True)
with torch.no_grad():
    with torch.enable_grad():
        y = x * 2
y.requires_grad

True

## Почему Backprop надо понимать

1. Backprop позволяет понимать, как те или иные операции, сложные конструкции в сети влияют на обнолвение весов.
Почему лучше сделать конкатенацию тензоров, а не поэлементное сложение. Для этого нужно посмотреть на backprop, как будут обновляться веса.

2. Даже на таком маленьком пример двуслойной MLP можно уже увидеть, когда `ReLU`, как функция активация, не очень хорошо применять. Если разреженные данные, то получить на выходе много нулей вероятнее, чем при использовании `LeakyReLU`, то есть градиенты будут нулевыми и веса никак не будут обновляться => сеть не обучается!

3. В архитектуре могут встречаться недифференцируемые операции, и первое - это нужно понять, потому что при обучении сети это может быть не сразу заметно, просто качество модели будет плохое, и точность хорошую не поулчится достичь.

Например, в одной из статей было предложено в качестве механизма внимания применить распределение Бернулли, которое умножается на выход промежуточного слоя сети. И эта опреация недифференцируема, нужно реализовывать backprop самим, тем самым обеспечить корректное протекание градиентов.


<img src="./images/Bernoulli.png" alt="8" style="width: 600px;"/>

Так же любая статья, которая предлагет новую целевую функцию для той или иной задачи, там всегда будут представлены градиенты, чтобы было понимание, как это влияет на обновление весов. Не просто так !

<img src="./images/BernoulliBackProp.png" alt="8" style="width: 600px;"/>

## nn.Module

В предыдущем примере архитектуру сети создавали используя последовательной способ объявления слоев сети -- `nn.Sequential`.

Но еще можно это сделать более гибким подходом:

In [23]:
class TwoLayerNet(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        """
        TwoLayerNet наследуется от nn.Module и тем самым полчаем возможность
        переопределять методы класса.
        В конструктуре создаем слои (обучаемые веса) и другие нужные перменные/функции,
        которые нужны для модели
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(input_size, hidden_size)
        self.linear2 = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        """
        Метод forward отвечает за прямое распростронение модели, 
        поэтому данный метод нужно переопределять обязательно, 
        чтобы задать логику прямого распростронения. 
        Именно в этот момент начинает строится динамический граф
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        
        return y_pred

In [25]:
batch_size = 64
input_size = 1000
hidden_size = 100
output_size = 10

x = torch.randn(batch_size, input_size, device=device, dtype=dtype)
y = torch.randn(batch_size, output_size, device=device, dtype=dtype)

model = TwoLayerNet(input_size, hidden_size, output_size)

loss_fn = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

for t in range(500):
    y_pred = model(x)

    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 2.2822232246398926
199 0.04613272100687027
299 0.0017005604458972812
399 8.421259553870186e-05
499 4.939337941323174e-06


# Домашние задание

1. Добавить Bias и посчитать для них градиенты.
2. Сравнить градинеты с тем, как считает PyTorch AutoGrad.

In [82]:
batch_size = 64
input_size = 3
hidden_size = 2
output_size = 1

In [83]:
# Create random input and output data
x = torch.randn(batch_size, input_size, device=device, dtype=dtype)
y = torch.randn(batch_size, output_size, device=device, dtype=dtype)

# Randomly initialize weights
w1 = torch.randn(input_size, hidden_size, device=device, dtype=dtype)
w2 = torch.randn(hidden_size, output_size, device=device, dtype=dtype)

bias1 = torch.randn(batch_size, output_size, device=device, dtype=dtype)
bias2 = torch.randn(batch_size, output_size, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y
    #TODO
    h_1 = x.mm(w1) + bias1
    h_relu = h_1.clamp(min=0)
    out = h_relu.mm(w2) + bias2
    
    # Compute and print loss
    loss = (out - y).pow(2).sum().item()    
    
    
    # Backward pass:
    dloss_dout = 2 * (out - y)
    
    grad_w2 = h_relu.t().mm(dloss_dout)
    
    grad_h_relu = dloss_dout.mm(w2.t())
    
    grad_h_relu[h_1 < 0] = 0
    
    grad_w1 = x.t().mm(grad_h_relu)
    
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2
    if t % 100 == 99:
        print('Loss on iteration {} = {}'.format(t, loss))

Loss on iteration 99 = 1063.1678466796875
Loss on iteration 199 = 913.5365600585938
Loss on iteration 299 = 796.2064819335938
Loss on iteration 399 = 703.3566284179688
Loss on iteration 499 = 628.15087890625


### PyTorch AutoGrad

In [79]:
# Create random input and output data
x = torch.randn(batch_size, input_size, device=device, dtype=dtype)
y = torch.randn(batch_size, output_size, device=device, dtype=dtype)

# Randomly initialize weights
w1 = torch.randn(input_size, hidden_size, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(hidden_size, output_size, device=device, dtype=dtype, requires_grad=True)

bias1 = torch.randn(batch_size, output_size, device=device, dtype=dtype, requires_grad=True)
bias2 = torch.randn(batch_size, output_size, device=device, dtype=dtype, requires_grad=True)

In [71]:
x.shape

torch.Size([64, 3])

In [72]:
y.shape

torch.Size([64, 1])

In [73]:
w1.shape

torch.Size([3, 2])

In [74]:
w2.shape

torch.Size([2, 1])

In [75]:
bias1.shape

torch.Size([64, 1])

In [76]:
(x.mm(w1) + bias1).shape

torch.Size([64, 2])

In [77]:
((x.mm(w1) + bias1).clamp(min=0).mm(w2) + bias2).shape

torch.Size([64, 1])

In [78]:
learning_rate = 1e-6
for t in range(500):
    y_pred = (x.mm(w1) + bias1).clamp(min=0).mm(w2) + bias2

    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())
    
    # Теперь подсчет градиентов для весов происходит при вызове backward
    loss.backward()
   
    # Обновляем значение весов, но укзаываем, чтобы PyTorch не считал эту операцию, 
    # которая бы учавствовала бы при подсчете градиентов в chain rule
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        bias1 -= learning_rate * bias1.grad
        bias2 -= learning_rate * bias2.grad
        
        # Теперь обнуляем значение градиентов, чтобы на следующем шаге 
        # они не учитывались при подсчете новых градиентов,
        # иначе произойдет суммирвоание старых и новых градиентов
        w1.grad.zero_()
        w2.grad.zero_()
        bias1.grad.zero_()
        bias2.grad.zero_()

99 223.76693725585938
199 216.8677978515625
299 210.6424560546875
399 205.01455688476562
499 199.87899780273438


In [81]:
import torch.optim as optim

loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6
optimizer = torch.optim.Adam([w1, w2, bias1, bias2], lr=learning_rate)

for t in range(500):
    optimizer.zero_grad()
    
    y_pred = (x.mm(w1) + bias1).clamp(min=0).mm(w2) + bias2
    
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
    
    loss.backward()
   
    optimizer.step()

99 126.3984146118164
199 126.36009979248047
299 126.32183837890625
399 126.28363037109375
499 126.24543762207031
